# 2.0 Model Training
<div style="text-align: center">
<img src="static/anvil_diagram.png" alt="Anvil diagram" width="500"/>  
</div>

## How the Anvil Infrastructure Works

Anvil is our primary infrastructure for model training and evaluation, built to support scalable, reproducible, and rigorous development of ADMET prediction models.  

Recognizing that building the best models requires training many variants, ensuring their reproducibility, and enabling robust performance comparisons, Anvil centers around a YAML-based recipe system.  

These recipes allow users to specify model architectures and training procedures in a standardized, shareable format—minimizing code duplication while supporting both deep learning and traditional machine learning approaches.  

Designed with both internal and external engagement in mind, Anvil aims to lower the barrier for outside users to adopt and fine-tune models by offering simple, transparent workflows. Long-term, it will serve as a foundation for broader community involvement and model reuse.

### Requirements
To run Anvil, you need:
1. A dataset that has been processed with `01_Curate_ChEMBL_Data.ipynb`.  
2. A `YAML` file with instructions for Anvil. We will show you how to create this file in this notebook.

## Training a Model to Predict CYP3A4 Inhibition

Now that we have a cleaned a dataset, we can train a model to predict **CYP3A4 inhibition**.  

This notebook will walk you through how to run the Anvil model training workflow with the CYP3A4 data processed and cleaned in previous notebooks.

## Creating the YAML file
The heart of an anvil run is in its `YAML` configuration file. Here we specify nearly everything needed to:
- load data
- preprocess it
- split the data appropriately into train/validation/test
- featurize according to model selection
- train the model
- and, finally, validate on the test set (which generates performance metrics and plots)  

We will walkthrough two `YAML` files: one for training a traditional machine learning model (`anvil_lgbm.yaml`) and one for training a deep learning model (`anvil_chemprop.yaml`).

## Training a Traditional Machine Learning model: LightGBM 

Here is a `YAML` file for training a LightGBM (LGBM) model. We are using the previously curated CYP3A4 data from ChEMBL. Be sure to read through the comments (in green) to understand each field.  

1. At a minimum, ensure `resource`, `input_col`, and `target_cols` are specified to match your dataset, as these will vary per dataset
2. The `procedure` section may not need much modification, especially if not tweaking parameters, but look it over to make sure it’s sensible

```yaml 
# This spection specifies the data that will be input into the model
data:
  # Specify the dataset file
  resource: ../../01_Data_Curation/processed_data/processed_CYP3A4_inhibition.csv
  type: intake
  input_col: OPENADMET_CANONICAL_SMILES
  # Specify each (1+) of the target columns, or the column that you're trying to predict
  target_cols:
  - OPENADMET_LOGAC50
  dropna: true

# Additional metadata
# This should be descriptive as the tags in these fields will annotate downstream Anvil processes:
# mainly, when you do model inference
metadata:
  authors: Your Name
  email: youremail@email.com
  biotargets:
  - CYP3A4
  build_number: 0
  description: basic regression using a LightGBM model
  driver: sklearn
  name: lgbm_pchembl
  tag: openadmet-chembl
  tags:
  - openadmet
  - test
  - pchembl
  version: v1

# Section specifying training procedure:
# What model will you use?
# What featurizers will the model use?
# What hyperparameters will the mdoel use?
procedure:
# Featurization specification
  feat:
    # Using concatenated features, which combines multiple featurizers
    # here we use DescriptorFeaturizer and FingerprintFeaturizer for 2D RDKit descriptors and ECFP4 fingerprints
    # See openadmet.models.features 
    type: FeatureConcatenator
    # Add parameters for the featurizer. Full description of the featurizer options are in Section 5.
    params:
      featurizers:
        DescriptorFeaturizer:
          descr_type: "desc2d"
        FingerprintFeaturizer:
          fp_type: "ecfp:4"
  
  # Model specification
  model:
    # Indicate model type
    # See openadmet.models.architecture for all model types
    type: LGBMRegressorModel
    # Specify model parameters
    params:
      alpha: 0.005
      learning_rate: 0.05
      n_estimators: 500


  # Specify data splits
  split:
    # Specify how data will be split
    # See openadmet.models.split
    type: ShuffleSplitter
    # Specify split parameters
    params:
      random_state: 42
      train_size: 0.8
      val_size: 0.0 # For LGBM, no validation set is needed
      test_size: 0.2 # If you want to compare tree-based models with Dl models later, the test sizes should match
    
  # Specify training configuration
  train:
    # Specify the trainer, here SKLearnBasicTrainer as model has an sklearn interface
    # could also use SKLearnGridSearchTrainer for hyperparameter tuning
    type: SKLearnBasicTrainer


# Section specifying report generation
# What cross validation splits will you use?
# You can also specify the min and max values of your plots
report:
  # Configure evaluation
  eval:
  # Generate regression metrics
  - type: RegressionMetrics
    params: {}
  # Generate regression plots & do cross validation
  - type: SKLearnRepeatedKFoldCrossValidation
    params:
      axes_labels:
      - True pAC50
      - Predicted pAC50
      max_val: 10
      min_val: 3
      pXC50: true
      n_splits: 5
      n_repeats: 5
      title: True vs Predicted pAC50 on test set

```

After you have created or modified this `YAML` file to your liking, you can run the workflow with the below command either in a `bash` cell or in your command line:
```
  openadmet anvil --recipe-path <your_file.yaml> --output-dir <output folder name>
```

This may take 5-10 minutes to run, depending on the number of epochs, your hyperparameters (e.g. learning rate), etc.

In [ ]:
%%bash
export OADMET_NO_RICH_LOGGING=1

openadmet anvil --recipe-path anvil_lgbm.yaml --output-dir lgbm

The outputs of the Anvil workflow are in `/anvil_training`:  
- `/data` folder includes the split data, saved as `.csv`
- `/recipe_components` folder contains the inputs from the `2.1_anvil_lgbm.yaml` file split by section
- `cross_validation_metrics.json` is the cross validation metrics of the model saved as a `.json` file
- `model.json` is the model's hyperparameters saved as a `.json` file
- `regression_metrics.json` is the regression metrics saved as a `.json` file
- `model.pkl` is the trained model saved as `.pkl` which can be loaded and used for predictions elsewhere
- `cross_validation_regplot.png` is a plot of the cross validation metrics of the model
- `anvil_recipe.yaml` is a copy of the input `.yaml`

Here are the results of above trained LGBM model:

<img src="lgbm/cross_validation_regplot.png" alt="LGBM model results" width="500"/>

## Training a Deep Learning model: ChemProp

Our framework is also capable of training deep learning models, but for ease of demonstration on CPU, we won't actually train the model here. We recommend training deep learning models on GPU.  

As an example, we've provided an already trained ChemProp model for you to look at for this demo.

Here is a `YAML` file (`anvil_chemprop.yaml`) for training OpenADMET's ChemProp model. We are using the same ChEMBL CYP3A4 dataset. Be sure to note the different fields required for deep learning.

```yaml
# This spection specifies the input data 
data:
  # Specify the dataset file
  resource: ../01_Data_Curation/processed_data/processed_CYP3A4_inhibition.csv
  type: intake
  input_col: OPENADMET_CANONICAL_SMILES
  # Specify each (1+) of the target columns, or the column that you're trying to predict
  target_cols:
  - OPENADMET_LOGAC50


# Additional metadata
metadata:
  authors: Your Name
  email: youremail@mail.com
  biotargets:
  - CYP3A4
  build_number: 0
  description: basic regression using a ChemProp multitask task model
  driver: pytorch
  name: chemprop_pchembl
  tag: chemprop-CYP3A4-chembl
  tags:
  - openadmet
  - test
  version: v1

# Section specifying training procedure
procedure:
  # Featurization specification
  feat:
    # Using the ChemPropFeaturizer (for ChemProp model)
    # See openadmet.models.features
    type: ChemPropFeaturizer
    # No parameters passed
    params: {}
  
  # Model specification
  model:
    # Indicate model type
    # See openadmet.models.architecture
    type: ChemPropModel
    # Specify model parameters
    params:
      depth: 4
      ffn_hidden_dim: 1024
      ffn_hidden_num_layers: 4
      message_hidden_dim: 2048
      dropout: 0.2
      batch_norm: True
      messages: bond
      n_tasks: 1 # Number of tasks should match the number of target columns
      from_chemeleon: False

  # Specify data splits
  split:
    # Specify how data will be split
    # See openadmet.models.split
    type: ShuffleSplitter
    # Specify split parameters
    params:
      random_state: 42
      train_size: 0.7
      val_size: 0.1
      test_size: 0.2
    
  # Specify training configuration
  train:
    # Specify the trainer, here LightningTrainer as ChemProp is a PyTorch Lightning model
    # See openadmet.models.trainer
    type: LightningTrainer
    # Specify model parameters
    params:
      accelerator: gpu
      early_stopping: true
      early_stopping_patience: 10
      early_stopping_mode: min
      early_stopping_min_delta: 0.001
      max_epochs: 50
      monitor_metric: val_loss
      use_wandb: false
      wandb_project: demos # Specify wandb project name according to guidelines

# Section specifying report generation
report:
  # Configure evaluation
  eval:
  # Generate regression metrics
  - type: RegressionMetrics
    params: {}
  # Generate regression plots & do cross validation
  - type: PytorchLightningRepeatedKFoldCrossValidation
    params:
      axes_labels:
      - True LogAC50
      - Predicted LogAC50
      n_repeats: 5
      n_splits: 5
      random_state: 42
      pXC50: true
      title: True vs Predicted LogAC50 on test set
```

The command is
```
openadmet anvil --recipe-path anvil_chemprop.yaml --output-dir chemprop
```
We recommend training deep learning models on GPU.

The results of a pre-trained version we provide are shown here


<img src="./chemprop/cross_validation_regplot.png" alt="ChemProp model results" width="500"/>

## Training a Multitask Deep Learning Model: ChemProp

Similarly to the single task deep learning example above, we've gone ahead and trained this model for you. We recommend training deep learning models on GPU.

There may be instances where you will want to train a model to predict compound activity on multiple protein targets.

For example, you may have endpoints that share a biochemical pathway such that activity on one is thought to be somewhat correlated to the other. 

It would thus be useful to train a multitask model on multiple targets. The `YAML` file example shown below is `anvil_multitask.yaml`.

```yaml
# Section specifying input data
data:
  # Specify the dataset file, can be S3 path etc.
  resource:  ../01_Data_Curation/processed_data/multitask.parquet
  # must be intake
  type: intake
  # Specify input column containing SMILES
  input_col: OPENADMET_CANONICAL_SMILES
  # Specify whether or not to drop NaN data rows
  dropna: False
  # Specify each (1+) of the target columns
  target_cols:
  - OPENADMET_LOGAC50_cyp2j2
  - OPENADMET_LOGAC50_cyp3a4
  - OPENADMET_LOGAC50_cyp1a2
  - OPENADMET_LOGAC50_pxr
  - OPENADMET_LOGAC50_cyp2d6
  - OPENADMET_LOGAC50_cyp2c9
  - OPENADMET_LOGAC50_ahr

# Additional metadata
metadata:
  authors: Your Name
  email: youremail@mail.com
  biotargets:
  - CYP3A4
  - CYP2J2
  - CYP1A2
  - CYP2D6
  - CYP2C9
  - PXR
  - AHR
  build_number: 0
  description: basic regression using a ChemProp multitask task model
  driver: pytorch
  name: chemprop_pchembl
  tag: chemprop
  tags:
  - openadmet
  - test
  - chemprop
  version: v1

# Section specifying training procedure
procedure:
  # Featurization specification
  feat:
    # Using the ChemPropFeaturizer (for ChemProp model)
    # See openadmet.models.features
    type: ChemPropFeaturizer
    # No parameters passed
    params: {}
  
  # Model specification
  model:
    # Indicate model type
    # See openadmet.models.architecture
    type: ChemPropModel
    # Specify model parameters
    params:
      depth: 4
      ffn_hidden_dim: 1024
      ffn_hidden_num_layers: 4
      message_hidden_dim: 2048
      dropout: 0.2
      batch_norm: True
      messages: bond
      n_tasks: 7 # Number of tasks should match the number of target columns
      from_chemeleon: False

  # Specify data splits
  split:
    # Specify how data will be split, can be ShuffleSplitter, ScaffoldSplitter, etc.
    # See openadmet.models.split
    type: ShuffleSplitter
    # Specify split parameters
    params:
      random_state: 42
      train_size: 0.7
      val_size: 0.1
      test_size: 0.2
    
  # Specify training configuration
  train:
    # Specify the trainer, here LightningTrainer as ChemProp is a PyTorch Lightning model
    # See openadmet.models.trainer
    type: LightningTrainer
    # Specify model parameters
    params:
      accelerator: gpu
      early_stopping: true
      early_stopping_patience: 10
      early_stopping_mode: min
      early_stopping_min_delta: 0.001
      max_epochs: 50
      monitor_metric: val_loss
      use_wandb: false
      wandb_project: demos # Specify wandb project name according to guidelines

# Section specifying report generation
report:
  # Configure evaluation
  eval:
  # Generate regression metrics
  - type: RegressionMetrics
    params: {}
  # Generate regression plots & do cross validation
  - type: PytorchLightningRepeatedKFoldCrossValidation
    params:
      axes_labels:
      - True LogAC50
      - Predicted LogAC50
      n_repeats: 5
      n_splits: 5
      random_state: 42
      pXC50: true
      title: Multitask True vs Predicted LogAC50 on test set
```

The results of a pre-trained version we provide are shown here


<img src="./multitask/cross_validation_regplot_OPENADMET_LOGAC50_cyp3a4.png" alt="ChemProp model results" width="500"/>

We will examine the full results of these models in `03_Model_Comparison`.

Congrats! You now know how to train models with the Anvil workflow. Explore our [model catalog](https://github.com/OpenADMET/openadmet-models/tree/main/openadmet/models) for other model architectures and featurizers.


**Now let's compare the performance of our models!**

~ End of `02_Training_Models` ~